In [1]:
import json #for json files
import sys #used to close json file
import asf_search as asf #ASF search API tool
from hyp3_sdk import HyP3 #ASF API tool for sending INSAR jobs
import csv #creating a .csv
import requests #for checking status of job

#import libraries for file management
import os 
from zipfile import ZipFile 
import shutil

import geopandas as gpd
import pandas as pd
from datetime import datetime
from datetime import timedelta

import psycopg2

In [8]:
conn = psycopg2.connect(host='localhost',database = 'postgres', user='postgres',password = 'postgrespw',port=32768)

In [3]:
directory = os.getcwd()

shp = gpd.GeoDataFrame.from_file('geodata/SC-Unit1D.shp')
geometry = str(shp['geometry'][0])
date = str(shp['Date'][0])
JobName = str(shp['Name'][0]).replace(" ","").replace("-","")
print(JobName)
d2 = datetime.strptime(date, '%Y-%m-%d')
end = str(d2 + timedelta(days = 1)*365)
start = str(d2 - timedelta(days = 1)*365)

Unit1D2019


In [65]:
jsonFile = open('geodata/Login.json')
login = json.load(jsonFile)

userName = login['UserName']
if userName == "":
    print("Please input Username")
    sys.exit()
pw = login['Password']
if pw =="":
    print("Please input Password")
    sys.exit()

jsonFile.close()

TypeError: 'NoneType' object is not callable

In [4]:
#Create an API search based on set parameters
searchResults = asf.search(
    #search parameters taken from filled out json file
    platform = "Sentinel-1",
    beamMode = "IW",
    polarization = "VV+VH",
    intersectsWith = geometry, 
    processingLevel= "SLC",
    start = start,
    end = end
)
print('Done')
data = searchResults.geojson()

C:\Users\Melay\AppData\Roaming\Python\Python310\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


Done


In [ ]:
data = searchResults.geojson()
print(data)

In [6]:
#Extracts scenes from job search and creates list of all scene names
def get_scene_name(): 
    count = len(data["features"])                   
    scenes = []
    dates = []
    for x in range(count):
        scene = data["features"][x]["properties"]["sceneName"]
        date = data["features"][x]["properties"]["startTime"]
        scenes.append(scene)
        dates.append(date)
    scenes.reverse()
    dates.reverse()
    df = pd.DataFrame(list(zip(dates, scenes)),columns =['Dates', 'SceneNames'])
    return df
scenesList= get_scene_name() #list with all scene names

In [ ]:
print(scenesList)

In [ ]:

filtered = scenesList.drop_duplicates()
scenes = list(filtered['SceneNames'])
dates = list(filtered['Dates'])
print(scenes)

In [ ]:
def create_jobs(data,date): 
    jobs = [] 
    count = len(data) 
    for i in range(count-1): 
        scene1 = data[i]
        scene2 = data[i+1]
        sceneDate = datetime.strptime(date[i+1], '%Y-%m-%dT%H:%M:%S.%f').date()
        print(sceneDate)
        print(len(scene2))
        insar_job = HyP3.prepare_insar_job(scene1, scene2, name = JobName, include_displacement_maps=True,include_dem=True,include_look_vectors=True) 
        print(insar_job)
        jobs.append(insar_job) 
        pair = scene1+','+scene2
        cursor = conn.cursor()
        cursor.execute("INSERT INTO project1 (pair, site, reference_date) VALUES(%s, %s, %s)", (pair, JobName, sceneDate))
        conn.commit()
        cursor.close()
    return jobs 
jobsList = create_jobs(scenes, dates)

#List of only 50 potential jobs for testing
print(jobsList[0:50])


In [ ]:
hyp3 = HyP3(username = userName, password = pw) #authenticate using ASF credentials
batch = hyp3.submit_prepared_jobs(prepared_jobs = jobsList)

#Creates New Folder called "data" where the jobs will be downloaded
output = os.path.join(directory, 'rawData')
if not os.path.exists(output):
    os.makedirs(output)

#Watch and Download job
if not batch.complete():
    batch = hyp3.watch(batch)
#Downloads files into data folder
batch.download_files(location = output)

In [11]:
#Creates List of zip file names
zipfiles = os.listdir(output)

#Unzips each zip file and deletes the zip
for x in range(len(zipfiles)):
    file = os.path.join(output, zipfiles[x])
    print(file)
    with ZipFile(file, 'r') as zip:
        zip.extractall(output)
    os.remove(file)

c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181006T001831_20181018T001831_VVP012_INT80_G_ueF_F50E.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181018T001831_20181030T001831_VVP012_INT80_G_ueF_0F0B.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181030T001831_20181111T001831_VVP012_INT80_G_ueF_AC52.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181111T001831_20181123T001831_VVP012_INT80_G_ueF_9372.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181123T001831_20181217T001830_VVP024_INT80_G_ueF_65E9.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181217T001830_20181229T001829_VVP012_INT80_G_ueF_4C33.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20181229T001829_20190110T001829_VVP012_INT80_G_ueF_DD12.zip
c:\Users\Melay\Documents\CoastalEnrichment\Code\rawData\S1AA_20190110T001829_20190122T001828_VVP012_INT80_G_ueF_BDDB.zip
c:\Users\Melay\Documents\Coastal

In [16]:
#Create List of unzipped folders
output = os.path.join(directory, 'rawData')
folderList = os.listdir(output)
vert_disp_list = []
#Loop through the folders
for x in range(len(folderList)):
    folderName = os.path.join(output, folderList[x])
    fileList = os.listdir(folderName)
    #Loop through files in folder
    for file in fileList:
        #Select file that ends in vert_disp.tif
        if file.endswith("vert_disp.tif"):
            fileName = os.path.join(folderName, file)
            vert_disp_list.append(fileName)
        if file.startswith(folderList[x]+'.txt'):
            fileName = os.path.join(folderName, file)
            f = open(fileName,'r')
            lines = f.readlines()
            scene1 = lines[0].split(' ')
            scene2 = lines[1].split(' ')
         
            pair = scene1[2].replace('\n','')+','+scene2[2].replace('\n','')
            print(pair)
            cursor = conn.cursor()
            cursor.execute("SELECT insar_name from project1 where insar_name = %s",(folderList[x],))
            row = cursor.fetchall()
            if row == []:
                cursor.execute("UPDATE project1 SET insar_name = %s where pair = %s",(folderList[x],pair))
                conn.commit()
            cursor.close()

#print(vert_disp_list)

S1A_IW_SLC__1SDV_20181006T001831_20181006T001858_024008_029F7D_A5FC,S1A_IW_SLC__1SDV_20181018T001831_20181018T001858_024183_02A534_24E3
S1A_IW_SLC__1SDV_20181018T001831_20181018T001858_024183_02A534_24E3,S1A_IW_SLC__1SDV_20181030T001831_20181030T001858_024358_02AAD0_3405
S1A_IW_SLC__1SDV_20181030T001831_20181030T001858_024358_02AAD0_3405,S1A_IW_SLC__1SDV_20181111T001831_20181111T001858_024533_02B121_5FA7
S1A_IW_SLC__1SDV_20181111T001831_20181111T001858_024533_02B121_5FA7,S1A_IW_SLC__1SDV_20181123T001831_20181123T001858_024708_02B792_F0D5
S1A_IW_SLC__1SDV_20181123T001831_20181123T001858_024708_02B792_F0D5,S1A_IW_SLC__1SDV_20181217T001830_20181217T001857_025058_02C3CD_BCEA
S1A_IW_SLC__1SDV_20181217T001830_20181217T001857_025058_02C3CD_BCEA,S1A_IW_SLC__1SDV_20181229T001829_20181229T001856_025233_02CA25_3D1A
S1A_IW_SLC__1SDV_20181229T001829_20181229T001856_025233_02CA25_3D1A,S1A_IW_SLC__1SDV_20190110T001829_20190110T001856_025408_02D072_3832
S1A_IW_SLC__1SDV_20190110T001829_20190110T001856

In [18]:
#Import library for file management
import os

#Libraries for dealing with geotifs
from osgeo import gdal,ogr
import rasterio as rio

#Math and plotting
import numpy as np
from scipy.ndimage import maximum_filter
import earthpy.plot as ep
import matplotlib.pyplot as plt

#Creating the gif
import imageio

In [34]:
print(vert_disp_list)

['c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181006T001831_20181018T001831_VVP012_INT80_G_ueF_F50E\\S1AA_20181006T001831_20181018T001831_VVP012_INT80_G_ueF_F50E_vert_disp.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181018T001831_20181030T001831_VVP012_INT80_G_ueF_0F0B\\S1AA_20181018T001831_20181030T001831_VVP012_INT80_G_ueF_0F0B_vert_disp.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181030T001831_20181111T001831_VVP012_INT80_G_ueF_AC52\\S1AA_20181030T001831_20181111T001831_VVP012_INT80_G_ueF_AC52_vert_disp.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181111T001831_20181123T001831_VVP012_INT80_G_ueF_9372\\S1AA_20181111T001831_20181123T001831_VVP012_INT80_G_ueF_9372_vert_disp.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181123T001831_20181217T001830_VVP024_INT80_G_ueF_65E9\\S1AA_20181123T001831_20181217T001830_VVP024_INT80_G_ueF_65E9_

In [36]:
dataset = rio.open(vert_disp_list[0])
boundingBox = dataset.bounds
left = boundingBox.left
right = boundingBox.right
top = boundingBox.top
bottom = boundingBox.bottom
print(boundingBox)
for x in range(1,len(vert_disp_list)):
    dataset = rio.open(vert_disp_list[x])
    boundingBox = dataset.bounds
    if boundingBox.left < left:
        left = boundingBox.left
    if boundingBox.right < right:
        right = boundingBox.right
    if boundingBox.top < top:
        top = boundingBox.top
    if boundingBox.bottom < bottom:
        bottom = boundingBox.bottom
    print(left,bottom,right,top)
minbbox = (left,bottom,right,top)
#outputBounds --- output bounds as (minX, minY, maxX, maxY) in target SRS

print(minbbox)

BoundingBox(left=260040.0, bottom=3214600.0, right=545720.0, top=3441560.0)
259880.0 3214600.0 545560.0 3441560.0
259880.0 3214600.0 545560.0 3441400.0
259880.0 3214600.0 545560.0 3441400.0
259880.0 3214600.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 3214440.0 545560.0 3441400.0
259880.0 321

In [39]:
#Crop the water mask incase it does not have the min bounds
#For loop to crop all the tif files to the min bounds
#all files need to be the same height and width to work with them in numpy
opts = gdal.WarpOptions(options=['tr'], outputBounds=minbbox, format="GTiff")
for geoTif in vert_disp_list:
    dataset = gdal.Open(geoTif)
    newfile = gdal.Warp(geoTif.split('.')[0]+'crop.tif', dataset , options=opts)
    dataset = None #close the file

In [40]:
croppedList = []
for x in range(len(folderList)):
    folderName = os.path.join(output, folderList[x])
    fileList = os.listdir(folderName)
    #Loop through files in folder
    for file in fileList:
        #Select file that ends in vert_disp.tif
        if file.endswith("crop.tif"):
            fileName = os.path.join(folderName, file)
            croppedList.append(fileName)
print(croppedList)

['c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181006T001831_20181018T001831_VVP012_INT80_G_ueF_F50E\\S1AA_20181006T001831_20181018T001831_VVP012_INT80_G_ueF_F50E_vert_dispcrop.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181018T001831_20181030T001831_VVP012_INT80_G_ueF_0F0B\\S1AA_20181018T001831_20181030T001831_VVP012_INT80_G_ueF_0F0B_vert_dispcrop.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181030T001831_20181111T001831_VVP012_INT80_G_ueF_AC52\\S1AA_20181030T001831_20181111T001831_VVP012_INT80_G_ueF_AC52_vert_dispcrop.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181111T001831_20181123T001831_VVP012_INT80_G_ueF_9372\\S1AA_20181111T001831_20181123T001831_VVP012_INT80_G_ueF_9372_vert_dispcrop.tif', 'c:\\Users\\Melay\\Documents\\CoastalEnrichment\\Code\\rawData\\S1AA_20181123T001831_20181217T001830_VVP024_INT80_G_ueF_65E9\\S1AA_20181123T001831_20181217T001830_VVP024_I